#**SCRAPPING DE LA WEB**: https://practicaldatascience.co.uk/data-science/how-to-scrape-and-parse-a-robotstxt-with-python

In [ ]:
from bs4 import BeautifulSoup
import requests
import lxml
import pandas as pd


url = 'https://www.fceia.unr.edu.ar/acustica/biblio/niveles.htm'

html_text = requests.get(url).text
soup = BeautifulSoup(html_text, 'lxml')

sound = soup.find('table')

In [ ]:
# HALLANDO EL TAMAÑO

info = requests.head(url)
print(info.headers)

"Content-Length': '5611"

{'Date': 'Sat, 24 Sep 2022 06:02:54 GMT', 'Server': 'Apache', 'Last-Modified': 'Tue, 28 Oct 2003 22:27:42 GMT', 'ETag': '"487a-3cac6c96f9380-gzip"', 'Accept-Ranges': 'bytes', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'X-Frame-Options': 'sameorigin', 'Content-Length': '5611', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Content-Type': 'text/html'}


"Content-Length': '5611"

In [ ]:
# HALLANDO EL TAMAÑO

info = requests.head(url)
print(info.headers)

{'Date': 'Sat, 24 Sep 2022 06:02:55 GMT', 'Server': 'Apache', 'Last-Modified': 'Tue, 28 Oct 2003 22:27:42 GMT', 'ETag': '"487a-3cac6c96f9380-gzip"', 'Accept-Ranges': 'bytes', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'X-Frame-Options': 'sameorigin', 'Content-Length': '5611', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Content-Type': 'text/html'}


In [ ]:
# HALLANDO EL TAMAÑO

info = requests.head(url)
print(info.headers)

{'Date': 'Sat, 24 Sep 2022 06:02:55 GMT', 'Server': 'Apache', 'Last-Modified': 'Tue, 28 Oct 2003 22:27:42 GMT', 'ETag': '"487a-3cac6c96f9380-gzip"', 'Accept-Ranges': 'bytes', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'X-Frame-Options': 'sameorigin', 'Content-Length': '5611', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Content-Type': 'text/html'}


##Eliminando las etiquetas !br! de la tabla para asegurar que las celdas correspondan 

In [ ]:
sound_str = str(sound).replace('<br/>', '').replace('<br>', '').replace('\r', '').replace('\n', '-')
sound = BeautifulSoup(sound_str, 'lxml')
sound_categorie = sound.find_all('tr')
sound_categories = [i.text.split('-') for i in sound_categorie]

### ***Aqui force a la bruta fuerza el cleansing del texto***

In [ ]:
categorias = map(lambda x: [element for element in x if len(element) > 0], sound_categories)
categorias = list(categorias)
for i in categorias:
    if len(i) > 3:
        i.pop(1)
#     print(f'{categorias.index(i)}-{(i)}: {len(i)}')

categorias[8].pop(2)
categorias[10].pop(1)
categorias[10].pop(3)
categorias[10][2] = 'Dificil uso del telefono'
categorias[13].pop(1)
categorias = categorias[1:]

for i in categorias:
    print(f'{categorias.index(i)}-{(i)}: {len(i)}')

0-['Sonidos característicos', 'Nivel de presión sonora [dB]', 'Efecto']: 3
1-['Zona de lanzamiento de cohetes', '180', 'Pérdida auditiva irreversible']: 3
2-['Operación en pista de jets', '140', 'Dolorosamente fuerte']: 3
3-['Trueno', '130', '\xa0']: 3
4-['Despegue de jets (60 m)', '120', 'Maximo esfuerzo vocal']: 3
5-['Martillo neumático', '110', 'Extremadamente fuerte']: 3
6-['Cami&oacuten recolector', '100', 'Muy fuerte']: 3
7-['Camión pesado (15 m)', '90', '   Daño auditivo (8 Hrs)']: 3
8-['Reloj Despertador (0,5 m)', '80', 'Molesto']: 3
9-['Restaurante ruidoso', '70', 'Dificil uso del telefono']: 3
10-['Aire acondicionado', '60', 'Intrusivo']: 3
11-['Tránsito de vehículos livianos ', '50', 'Silencio']: 3
12-['Líving', '40', '\xa0']: 3
13-['Biblioteca', '30', 'Muy silencioso']: 3
14-['Estudio de radiodifusión', '20', '\xa0']: 3
15-['\xa0', '10', 'Apenas audible ']: 3
16-['\xa0', '0', 'Umbral auditivo']: 3


#### Creando el DataFrame con pandas, pues ya corregí el texto

In [ ]:
df = pd.DataFrame(categorias)
df.columns = df.iloc[0]
df.drop(0, inplace=True)
df

,Sonidos característicos,Nivel de presión sonora [dB],Efecto
1,Zona de lanzamiento de cohetes,180,Pérdida auditiva irreversible
2,Operación en pista de jets,140,Dolorosamente fuerte
3,Trueno,130,
4,Despegue de jets (60 m),120,Maximo esfuerzo vocal
5,Martillo neumático,110,Extremadamente fuerte
6,Cami&oacuten recolector,100,Muy fuerte
7,Camión pesado (15 m),90,Daño auditivo (8 Hrs)
8,"Reloj Despertador (0,5 m)",80,Molesto
9,Restaurante ruidoso,70,Dificil uso del telefono
10,Aire acondicionado,60,Intrusivo


In [ ]:
#dB Value        180        140          130         120         110         100         90        80          70            60           50         40         30          20        10      0
df['RGB'] = ["178,34,34","178,34,34","178,34,34","178,34,34","178,34,34","160,82,45","255,0,0","255,69,0","255,105,180","218,165,32","255,215,0","0,100,0","173,255,47","127,255,0","127,255,0","127,255,0"]

In [ ]:
df

,Sonidos característicos,Nivel de presión sonora [dB],Efecto,RGB
1,Zona de lanzamiento de cohetes,180,Pérdida auditiva irreversible,"178,34,34"
2,Operación en pista de jets,140,Dolorosamente fuerte,"178,34,34"
3,Trueno,130,,"178,34,34"
4,Despegue de jets (60 m),120,Maximo esfuerzo vocal,"178,34,34"
5,Martillo neumático,110,Extremadamente fuerte,"178,34,34"
6,Cami&oacuten recolector,100,Muy fuerte,"160,82,45"
7,Camión pesado (15 m),90,Daño auditivo (8 Hrs),"255,0,0"
8,"Reloj Despertador (0,5 m)",80,Molesto,"255,69,0"
9,Restaurante ruidoso,70,Dificil uso del telefono,"255,105,180"
10,Aire acondicionado,60,Intrusivo,"218,165,32"


In [ ]:
df = df.sort_index(ascending=False)
df.to_csv("table_dB.txt", index=False, sep="|")

In [ ]:
df.sort_index(ascending=False)

,Sonidos característicos,Nivel de presión sonora [dB],Efecto,RGB
16,,0,Umbral auditivo,"127,255,0"
15,,10,Apenas audible,"127,255,0"
14,Estudio de radiodifusión,20,,"127,255,0"
13,Biblioteca,30,Muy silencioso,"173,255,47"
12,Líving,40,,"0,100,0"
11,Tránsito de vehículos livianos,50,Silencio,"255,215,0"
10,Aire acondicionado,60,Intrusivo,"218,165,32"
9,Restaurante ruidoso,70,Dificil uso del telefono,"255,105,180"
8,"Reloj Despertador (0,5 m)",80,Molesto,"255,69,0"
7,Camión pesado (15 m),90,Daño auditivo (8 Hrs),"255,0,0"
